In [293]:
from keras import backend as K
import tensorflow as tf

import numpy as np
import keras
from keras.preprocessing.text import Tokenizer

#TEXT
text_decoupe = [["The sun is shining in June! It is beautiful.","September is grey.","Life is beautiful in August.",
         "I like it","This and other things?"],
        ["The sun is shining in Fevrier! It is beautiful.","Decembre is grey.","Life is beautiful in Monday.",
         "I like it","This and other things?"],
        ["The sun is shining in Tuesday! It is beautiful.","Juin is grey.","Life is beautiful in August.",
         "I like it","This and other things?"]]
text = ["The sun is shining in June! It is beautiful.","September is grey.","Life is beautiful in August.",
         "I like it","This and other things?", "The sun is shining in Fevrier! It is beautiful.","Decembre is grey.",
         "Life is beautiful in Monday.","I like it","This and other things?",
         "The sun is shining in Tuesday! It is beautiful.","Juin is grey.","Life is beautiful in August.",
         "I like it","This and other things?"]

#TOKENIZER
token = keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                           lower=True, split=" ", char_level=False)
token.fit_on_texts(text)
data = np.zeros((len(text_decoupe),5,len(token.word_counts)))
for i in range(0,3):
    data[i,:,:] = token.texts_to_matrix(text_decoupe[i])[:,1:]
    
#EMBEDDING
input_dim = token.word_counts
embedding_output_dim = 10
input_text = keras.layers.Input(shape =(5,len(input_dim),), name = "main")
embedding_layer = keras.layers.Embedding(len(input_dim), embedding_output_dim, input_length = None,
                                        embeddings_initializer='uniform',embeddings_regularizer=None, 
                                        activity_regularizer=None,embeddings_constraint=None,
                                        mask_zero=False)(input_text)

#BAG_OF_WORDS
#bag = keras.layers.GlobalAveragePooling1D()(embedding_layer)
bag = keras.layers.Lambda(lambda x: K.sum(x, axis=2), output_shape=(5,10,))(embedding_layer)


#INPUTBIS
A = np.random.rand(3,5,4)
input_incise = keras.layers.Input((5,4,), name = "aux")

#MERGING
merged = keras.layers.concatenate([bag, input_incise])

#LSTM
units = 6
lstm = keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                  kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                  unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
                  activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,
                  dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=True, return_state=False,
                  go_backwards=False, stateful=False, unroll=False) (merged)


model = keras.Model(inputs = [input_text, input_incise] , outputs = lstm)
#print(model.predict([data,A]))
print(model.predict([data,A]).shape)

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

(3, 5, 6)


In [313]:
model.fit([data,A], Y)

Epoch 1/1
3/3 [==============================] - 0s 2ms/step - loss: 4.3088 - acc: 0.1333


In [312]:
#ONEHOT_ENCODING
from sklearn.preprocessing import OneHotEncoder
entree = [[2,1,2,1,2],[1,2,3,4,5],[3,4,0,0,0]]
nb_personnages = 7
enc = OneHotEncoder(n_values = nb_personnages)

enc.fit(entree)
Y = enc.transform(entree).toarray().reshape(3,5,7)[:,:,1:]